# VII — Vehicle Tracking — Robust Kalman Filtering Case Study

We track a moving vehicle in 2D from noisy position measurements.

- State: position + velocity in x/y  
- Dynamics: linear (discrete-time) with unknown input (drive force) and damping  
- Noise: mostly mild Gaussian, but with occasional large outliers  

A classical Kalman smoother corresponds to a least-squares problem.  
To reduce sensitivity to outliers, we replace the quadratic measurement penalty with a Huber loss, yielding a convex problem solved with CVXPY.

> Adapted from the official CVXPY example “Robust Kalman filtering for vehicle tracking”.

[Kalman Filter Explained](https://www.youtube.com/watch?v=IFeCIbljreY)

https://www.youtube.com/watch?v=R63dU5w_djQ

## Theory and Pre-requisite Knowledge

### Recursive Least Squares

Suppose we observe a sequence of data pairs $(\phi_t, y_t)$, where
$y_t \in \mathbb{R}$ is a scalar measurement and
$\phi_t \in \mathbb{R}^d$ is a known feature (regressor) vector.
We assume a linear measurement model

$$
y_t = \phi_t^\top \theta + \epsilon_t,
\qquad
\epsilon_t \sim \mathcal{N}(0,\sigma^2),
$$

where $\theta \in \mathbb{R}^d$ is an unknown parameter vector to be estimated.

A standard approach is least squares, which estimates $\theta$ by minimizing the sum of squared residuals over all observed data. However, recomputing a batch least-squares solution every time a new data point arrives is computationally expensive and impractical for online or real-time applications. Recursive least squares addresses this issue by updating the parameter estimate incrementally as new measurements become available, without revisiting the full data history. A common formulation is exponentially weighted least squares:

$$
\hat\theta_t
=
\arg\min_\theta
\sum_{k=1}^t \lambda^{\,t-k}\,
\big(y_k - \phi_k^\top \theta\big)^2,
\qquad
0 < \lambda \le 1,
$$

where $\lambda$ is a forgetting factor that discounts older data. Values of $\lambda$ close to 1 emphasize long-term consistency, while smaller values allow the estimator to adapt more quickly to changing parameters. The RLS algorithm produces a recursive update of the parameter estimate:

$$
\hat\theta_t
=
\hat\theta_{t-1}
+
K_t
\big(y_t - \phi_t^\top \hat\theta_{t-1}\big),
$$

where the term

$$
y_t - \phi_t^\top \hat\theta_{t-1}
$$

is the prediction error (or residual), measuring how well the current estimate explains the new observation. The gain vector $K_t$ determines how strongly the estimate should be corrected in response to this residual:

$$
K_t
=
\frac{P_{t-1}\phi_t}
{\lambda + \phi_t^\top P_{t-1}\phi_t}.
$$


Here, $P_t \in \mathbb{R}^{d \times d}$ is a symmetric positive-definite matrix that quantifies the uncertainty in the parameter estimate. When uncertainty is large or the new data is highly informative, the correction is larger; when uncertainty is small, updates are more conservative.

RLS is particularly useful in online parameter estimation problems, such as system identification, adaptive control, and signal processing, where data arrives sequentially and model parameters may change slowly over time. The central idea behind RLS is to maintain both an estimate of the unknown parameters and a measure of confidence in that estimate, updating both efficiently as new information becomes available.



## Kalman filter: from recursive least squares to dynamic state estimation

Recursive least squares is designed to estimate unknown but essentially static parameters from sequential data. While it is effective for online regression and system identification, it does not explicitly model how the quantity being estimated evolves over time. Any temporal variation must be handled indirectly through mechanisms such as forgetting factors. In many real-world problems, however, the unknown quantity is not static. The state of a physical system—such as the position and velocity of a car—changes continuously according to known physical laws, while also being subject to random disturbances. In these settings, we need an estimator that explicitly accounts for system dynamics rather than treating each observation as an independent regression problem.

The Kalman filter addresses this limitation by introducing a motion model that describes how the state evolves from one time step to the next, together with a measurement model that links the hidden state to noisy sensor observations. Both models are assumed to be linear, and uncertainty is modeled using Gaussian noise.

The objective of the Kalman filter is to estimate the hidden state of a dynamical system by optimally combining:

1) predictions generated by the motion model, and  
2) information provided by noisy measurements.

Like recursive least squares, the Kalman filter operates recursively and maintains both an estimate and an uncertainty matrix. However, unlike RLS, it explicitly propagates uncertainty through time using the system dynamics and corrects predictions using new measurements. This allows the filter to track time-varying states in a principled and computationally efficient manner.

Under linear dynamics and Gaussian noise assumptions, the Kalman filter is optimal in the sense that it minimizes the mean squared estimation error and produces the posterior mean and covariance of the system state at each time step.

### State-space model
We model a discrete-time linear dynamical system as

$$
\begin{aligned}
\text{Motion model (State Equation):}\quad
x_{t+1} &= A x_t + B u_t + w_t, \\
\text{Measurement model (Observation Equation):}\quad
y_t &= C x_t + v_t,
\end{aligned}
\qquad t = 0,\dots,N-1.
$$

The motion model describes how the system state evolves from one time step to the next, while the measurement model specifies how the hidden state gives rise to observable sensor readings.

State and observations: In a typical vehicle tracking example, the state vector is $x_t = (p_x, p_y, v_x, u_y) \in \mathbb{R}^4$ where $(p_x, p_y)$ denotes position and $(u_x, u_y)$ denotes velocity. The measurement vector is $y_t \in \mathbb{R}^2$  representing observed position data, for example from a GPS sensor.

Inputs and noise: The model includes both control inputs and stochastic disturbances:

- $u_t$ is a known control input, such as commanded acceleration.  
- $w_t$ is process (motion) noise, capturing unmodeled dynamics and random disturbances. It is typically modeled as $w_t \sim \mathcal{N}(0, Q)$ where $Q$ is the process noise covariance.
- $v_t$ is measurement noise, capturing sensor inaccuracies, modeled as $v_t \sim \mathcal{N}(0, R)$ where $R$ is the measurement noise covariance.

Interpretation of system matrices:
- $A$ is the state transition matrix; it describes the deterministic evolution of the state.
- $B$ maps the control input $u_t$ into the state.
- $C$ maps the state to the measurement space, indicating which components of the state are observable.
- $Q$ and $R$ quantify uncertainty in the motion and measurement models, respectively.

Importantly, $A$, $B$, and $C$ describe system structure, not uncertainty; only $Q$ and $R$ are covariance matrices.

### What is being estimated?

At each time step $t$, the Kalman filter maintains two coupled quantities:

- $\hat x_{t+1|t}$: the estimate of the state after incorporating measurements up to time $t$,
- $P_{t|t}$: the covariance of the estimation error,
  $$
  P_{t|t}
  =
  \mathbb{E}\big[(x_t - \hat x_{t|t})(x_t - \hat x_{t|t})^\top\big].
  $$

These quantities summarize all past information relevant for future estimation.

The recursion is initialized with a prior belief

$$
x_0 \sim \mathcal{N}(\hat x_{0|0}, P_{0|0}),
$$

which encodes initial knowledge and uncertainty about the system state.

## The two-step recursion

As in recursive least squares, estimation proceeds by alternating between a prediction based on the current model and a correction driven by new data. However, here both the estimate and its uncertainty are explicitly propagated through the system dynamics.

### Step 1: Prediction (time update)

Using the motion model, the filter predicts the next state before observing $y_t$:

$$
\hat x_{t|t-1} = A \hat x_{t-1|t-1} + B u_{t-1}.
$$

Uncertainty is propagated forward as

$$
P_{t|t-1} = A P_{t-1|t-1} A^\top + Q.
$$

This step extrapolates the current belief through the dynamics and increases uncertainty to account for process noise.

### Step 2: Update (measurement correction)

Once a new measurement arrives, the filter computes the innovation (or surprise term):

$$
r_t = y_t - C \hat x_{t|t-1},
$$

which measures the discrepancy between the observed measurement and its predicted value.

The uncertainty associated with this discrepancy is

$$
S_t = C P_{t|t-1} C^\top + R.
$$

The Kalman gain is then defined as

$$
K_t = P_{t|t-1} C^\top S_t^{-1}.
$$

This gain determines how much the prediction should be corrected using the new measurement.

The state estimate is updated according to

$$
\hat x_{t|t} = \hat x_{t|t-1} + K_t r_t,
$$

and the corresponding uncertainty is reduced as

$$
P_{t|t} = (I - K_t C) P_{t|t-1}.
$$

### Interpretation

- Large measurement noise (large $R$) leads to a smaller Kalman gain, causing the filter to rely more heavily on the model prediction.
- Large prediction uncertainty (large $P_{t|t-1}$) leads to a larger Kalman gain, causing the filter to rely more heavily on the measurement.

Thus, the Kalman filter automatically balances model-based prediction and data-driven correction in an uncertainty-aware manner, extending the core ideas of recursive least squares to time-evolving dynamical systems.


## 1. Problem setup  

Give breief introd to Recursive leastime qaure estimator?

Kalman alos include a motion model

We model a discrete-time linear dynamical system:

$$
\begin{aligned}
{Motion Model}:\\
x_{t+1} &= A x_t + B u_t + w_{t} \\
{Measurement Model}:\\
y_t &= C x_t + v_t,
\end{aligned}
\qquad t=0,\dots,N-1
$$

- $x_t \in \mathbb{R}^4$: state $=(p_x,p_y,u_x,u_y)$ (position + velocity)  
- $u_t \in \mathbb{R}^2$: unknown input (drive force in x/y)  
- $w_t \in \mathbb{R}^2$: Process or Mitoion noise $2_t ~Normal (0, Q_t)$
- $y_t \in \mathbb{R}^2$: observed position (Measuring insturement such as GPS)
- $v_t \in \mathbb{R}^2$: measurement noise  $v ~Norma (0, R_t$)$
- A is covraiance matric?
- B?
- C?


Question: how to best estimate the posiiton of car using initial estiamte and measurment both of whcih are noise?
Kalman filter is an algorithm to estimate the state of the system using past and possibley observations and current and possibly noisy measurments of that systement - more specidfically?

Its a two step process:
1. Pedictition step
   $X^hat_t+1 = A x_t + B_u_t$
   $
2. Update step

Suprise term?
Kalman Ratio?

Oprimal estimation algorithm
### Standard (quadratic) Kalman smoothing

$$
\begin{aligned}
\min_{\{x_t,w_t,v_t\}} &\sum_{t=0}^{N-1} \left(\|w_t\|_2^2 + \tau\,\|v_t\|_2^2\right) \\
\text{s.t. } &x_{t+1}=Ax_t + Bw_t,\quad y_t=Cx_t + v_t.
\end{aligned}
$$

### Robust Kalman smoothing (Huber measurements)

Define the Huber penalty
$$
\phi_\rho(a)=
\begin{cases}
\|a\|_2^2, & \|a\|_2\le \rho \\
2\rho\|a\|_2 - \rho^2, & \|a\|_2 > \rho
\end{cases}
$$

and solve
$$
\min \sum_{t=0}^{N-1}\left(\|w_t\|_2^2 + \tau\,\phi_\rho(v_t)\right)
\quad \text{s.t. dynamics + measurement constraints.}
$$

Interpretation: small residuals are penalized quadratically, but large residuals only *linearly*, reducing outlier influence.

In [ ]:
# --- Imports & reproducibility ---
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

SEED_MAIN = 6
SEED_OUTLIERS = 0
np.random.seed(SEED_MAIN)

import sys, platform
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
print("NumPy:", np.__version__)
print("CVXPY:", cp.__version__)

## 2. Helper plotting functions

In [ ]:
import matplotlib

def plot_state(ts, actual, estimated=None, suptitle=None):
    """Plot position, velocity, and input over time in x and y."""
    trajectories = [actual]
    labels = ["True"]
    if estimated is not None:
        trajectories.append(estimated)
        labels.append("Estimated")

    fig, ax = plt.subplots(3, 2, sharex='col', sharey='row', figsize=(10, 8))
    for (x, w), lab in zip(trajectories, labels):
        ax[0,0].plot(ts, x[0,:-1], label=lab)
        ax[0,1].plot(ts, x[1,:-1], label=lab)
        ax[1,0].plot(ts, x[2,:-1], label=lab)
        ax[1,1].plot(ts, x[3,:-1], label=lab)
        ax[2,0].plot(ts, w[0,:], label=lab)
        ax[2,1].plot(ts, w[1,:], label=lab)

    ax[0,0].set_ylabel('x position')
    ax[1,0].set_ylabel('x velocity')
    ax[2,0].set_ylabel('x input')
    ax[0,1].set_ylabel('y position')
    ax[1,1].set_ylabel('y velocity')
    ax[2,1].set_ylabel('y input')

    for r in range(3):
        ax[r,1].yaxis.tick_right()
        ax[r,1].yaxis.set_label_position("right")

    ax[2,0].set_xlabel('time')
    ax[2,1].set_xlabel('time')

    ax[0,0].legend(loc="best")
    if suptitle:
        fig.suptitle(suptitle, y=1.02)
    plt.tight_layout()
    plt.show()

def plot_positions(traj, labels, axis=None):
    """Point clouds for (x,y) positions."""
    matplotlib.rcParams.update({'font.size': 12})
    n = len(traj)
    fig, ax = plt.subplots(1, n, sharex=True, sharey=True, figsize=(4*n, 4))
    if n == 1:
        ax = [ax]
    for i, x in enumerate(traj):
        ax[i].plot(x[0,:], x[1,:], 'o', alpha=.15, markersize=4)
        ax[i].set_title(labels[i])
        ax[i].set_xlabel("x")
        if i == 0:
            ax[i].set_ylabel("y")
        if axis is not None:
            ax[i].axis(axis)
    plt.tight_layout()
    plt.show()

## 3. Generate synthetic tracking data (with outliers)

We simulate $N$ timesteps. The vehicle starts at the origin with zero velocity.

- Inputs $w_t$ are standard Gaussian.
- Measurement noise $v_t$ is standard Gaussian except that a fraction $p$ of timesteps are outliers with a much larger standard deviation $\sigma$.

In [ ]:
# --- Problem size (change N if you want faster/slower solves) ---
N = 600              # timesteps
T = 50               # time horizon (seconds)
ts, delt = np.linspace(0, T, N, endpoint=True, retstep=True)

gamma = 0.05         # damping (0 = no damping)

# Dynamics matrices (state: [px, py, vx, vy])
A = np.zeros((4, 4))
B = np.zeros((4, 2))
C = np.zeros((2, 4))

A[0,0] = 1
A[1,1] = 1
A[0,2] = (1 - gamma*delt/2) * delt
A[1,3] = (1 - gamma*delt/2) * delt
A[2,2] = 1 - gamma*delt
A[3,3] = 1 - gamma*delt

B[0,0] = delt2 / 2
B[1,1] = delt2 / 2
B[2,0] = delt
B[3,1] = delt

C[0,0] = 1
C[1,1] = 1

# Noise / outlier settings
sigma_outlier = 20.0
p_outlier = 0.20

# Simulate
np.random.seed(SEED_MAIN)
x_true = np.zeros((4, N+1))
x_true[:, 0] = [0, 0, 0, 0]
y = np.zeros((2, N))

w_true = np.random.randn(2, N)          # drive force
v = np.random.randn(2, N)               # nominal measurement noise

# Inject outliers in the measurement noise
np.random.seed(SEED_OUTLIERS)
outlier_mask = (np.random.rand(N) <= p_outlier)
v[:, outlier_mask] = sigma_outlier * np.random.randn(2, N)[:, outlier_mask]

# Roll forward
for t in range(N):
    y[:, t] = C @ x_true[:, t] + v[:, t]
    x_true[:, t+1] = A @ x_true[:, t] + B @ w_true[:, t]

print(f"N={N}, outliers={outlier_mask.mean()*100:.1f}%")

In [ ]:
plot_positions(
    traj=[x_true[:2, :-1], y],
    labels=["True positions", "Noisy measurements"],
    axis=[-4, 14, -5, 20]
)

## 4. Standard Kalman smoothing as least squares (CVXPY)

We solve the quadratic objective (least squares). This tends to “chase” outliers because large residuals are penalized quadratically.

In [ ]:
def solve_kalman_least_squares(A, B, C, y, tau=0.08, solver="OSQP", verbose=False):
    """Quadratic Kalman smoothing as a QP."""
    n = y.shape[1]
    x = cp.Variable((4, n+1))
    w = cp.Variable((2, n))
    v = cp.Variable((2, n))

    obj = cp.Minimize(cp.sum_squares(w) + tau * cp.sum_squares(v))

    constr = []
    for t in range(n):
        constr += [
            x[:, t+1] == A @ x[:, t] + B @ w[:, t],
            y[:, t]   == C @ x[:, t] + v[:, t],
        ]

    prob = cp.Problem(obj, constr)
    try:
        prob.solve(solver=solver, verbose=verbose)
    except Exception:
        prob.solve(verbose=verbose)

    return np.array(x.value), np.array(w.value), prob.value, prob.status

x_kf, w_kf, obj_kf, status_kf = solve_kalman_least_squares(A, B, C, y, tau=0.08, solver="OSQP", verbose=False)
print("status:", status_kf)
print("objective:", obj_kf)

In [ ]:
plot_state(ts, (x_true, w_true), (x_kf, w_kf), suptitle="Standard (quadratic) Kalman smoothing")
plot_positions([x_true[:2, :-1], x_kf[:2, :-1]], ["True positions", "KF recovery"], axis=[-4, 14, -5, 20])

## 5. Robust Kalman smoothing (Huber measurements)

We replace the quadratic measurement penalty with a Huber penalty on the residual norm $\|v_t\|_2$.

In CVXPY:
`huber(norm(v[:,t]), rho)`

In [ ]:
def solve_kalman_robust_huber(A, B, C, y, tau=2.0, rho=2.0, solver="SCS", verbose=False):
    """Robust Kalman smoothing with Huber measurement loss (conic program)."""
    n = y.shape[1]
    x = cp.Variable((4, n+1))
    w = cp.Variable((2, n))
    v = cp.Variable((2, n))

    huber_terms = [tau * cp.huber(cp.norm(v[:, t], 2), rho) for t in range(n)]
    obj = cp.Minimize(cp.sum_squares(w) + cp.sum(huber_terms))

    constr = []
    for t in range(n):
        constr += [
            x[:, t+1] == A @ x[:, t] + B @ w[:, t],
            y[:, t]   == C @ x[:, t] + v[:, t],
        ]

    prob = cp.Problem(obj, constr)
    try:
        prob.solve(solver=solver, verbose=verbose)
    except Exception:
        prob.solve(verbose=verbose)

    return np.array(x.value), np.array(w.value), prob.value, prob.status

x_rkf, w_rkf, obj_rkf, status_rkf = solve_kalman_robust_huber(A, B, C, y, tau=2.0, rho=2.0, solver="SCS", verbose=False)
print("status:", status_rkf)
print("objective:", obj_rkf)

In [ ]:
plot_state(ts, (x_true, w_true), (x_rkf, w_rkf), suptitle="Robust Kalman smoothing (Huber measurements)")
plot_positions([x_true[:2, :-1], x_rkf[:2, :-1]], ["True positions", "Robust KF recovery"], axis=[-4, 14, -5, 20])

## 6. Quantitative comparison

We compare RMSE of position and velocity:

- Position RMSE uses $(p_x,p_y)$
- Velocity RMSE uses $(u_x,u_y)$

In [ ]:
def rmse(a, b):
    return np.sqrt(np.mean((a - b)2))

pos_true = x_true[:2, :-1]
vel_true = x_true[2:, :-1]

pos_kf = x_kf[:2, :-1]
vel_kf = x_kf[2:, :-1]

pos_rkf = x_rkf[:2, :-1]
vel_rkf = x_rkf[2:, :-1]

metrics = {
    "KF_pos_RMSE": rmse(pos_kf, pos_true),
    "RKF_pos_RMSE": rmse(pos_rkf, pos_true),
    "KF_vel_RMSE": rmse(vel_kf, vel_true),
    "RKF_vel_RMSE": rmse(vel_rkf, vel_true),
}
metrics

## 7. Residual diagnostics (who is “chasing” outliers?)

The estimated measurement residual is $\hat v_t = y_t - C\hat x_t$.  
We compare residual norms for quadratic vs robust smoothing and mark outlier timesteps.

In [ ]:
def residual_norms(C, x_hat, y):
    v_hat = y - (C @ x_hat[:, :-1])
    return np.linalg.norm(v_hat, axis=0)

r_kf = residual_norms(C, x_kf, y)
r_rkf = residual_norms(C, x_rkf, y)

plt.figure(figsize=(10, 3))
plt.plot(ts, r_kf, label="KF residual norm", alpha=0.9)
plt.plot(ts, r_rkf, label="Robust KF residual norm", alpha=0.9)
plt.scatter(ts[outlier_mask], r_kf[outlier_mask], s=12, alpha=0.6, label="Outlier timesteps (KF)")
plt.xlabel("time")
plt.ylabel(r"$\|y_t - C\hat x_t\|_2$")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

## 8. Hyperparameters ($\tau$, $\rho$): quick sweep

- Larger $	au$ forces the model to fit measurements more (less smoothing).
- Smaller $
ho$ makes the Huber loss switch to linear earlier (more robust).

We do a tiny sweep (keep it small to avoid long runtimes).

In [ ]:
taus = [0.5, 1.0, 2.0, 4.0]
rhos = [0.5, 1.0, 2.0]

results = []
for tau in taus:
    for rho in rhos:
        x_hat, w_hat, obj, status = solve_kalman_robust_huber(A, B, C, y, tau=tau, rho=rho, solver="SCS", verbose=False)
        pos_rmse = rmse(x_hat[:2, :-1], pos_true)
        vel_rmse = rmse(x_hat[2:, :-1], vel_true)
        results.append((tau, rho, pos_rmse, vel_rmse, status))

results_sorted = sorted(results, key=lambda t: t[2])
for tau, rho, prmse, vrmse, st in results_sorted[:8]:
    print(f"tau={tau:>4}, rho={rho:>3} | pos_RMSE={prmse:.3f}, vel_RMSE={vrmse:.3f} | status={st}")

## 9. Takeaways

1. Standard quadratic smoothing is optimal under Gaussian noise, but outliers can warp the estimate.
2. A Huber measurement penalty is a clean convex way to robustify the estimator.
3. CVXPY makes it easy to switch between models by changing only the objective.

### Exercises (good for reports / assignments)
- Set `p_outlier = 0` and verify quadratic and robust behave similarly.
- Increase `sigma_outlier` and see when the quadratic model collapses.
- Replace the Huber penalty with an $\ell_1$ penalty on $v_t$ and compare.
- Try different solvers (e.g., ECOS if available) and compare runtime/accuracy.